In [37]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [79]:
df = pd.read_csv("../data/raw/all_v2.csv")
df.columns

Index(['price', 'date', 'time', 'geo_lat', 'geo_lon', 'region',
       'building_type', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'object_type'],
      dtype='object')

In [53]:
df.shape

(5477006, 13)

In [80]:
REGION_ID = 2661

In [82]:
def select_region(df) -> pd.DataFrame:
    """
    Function selects the listings belonging to a specified city.
    :param input_path:
    :param output_path:
    :param region:
    :return:
    """
    df = df[df['region'] == REGION_ID]
    df.drop('region', axis=1, inplace=True)
    print(f'Selected {len(df)} samples in region {REGION_ID}.')

    return df

In [83]:
df = select_region(df)

Selected 461820 samples in region 2661.


In [84]:
df.head(2)

,price,date,time,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,2018-02-19,20:00:21,59.805808,30.376141,1,8,10,3,82.6,10.8,1
7,3600000,2018-03-04,20:52:38,59.875526,30.395457,1,2,5,1,31.1,6.0,1


In [89]:
df.describe()

,price,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type
count,4.618200e+05,461820.000000,461820.000000,461820.000000,461820.000000,461820.000000,461820.000000,461820.000000,461820.000000,461820.000000
mean,8.292682e+06,59.932848,30.319601,1.948281,7.582619,14.336625,1.829986,58.582237,12.749967,5.271231
std,1.411196e+07,0.084615,0.120120,0.954559,5.694418,7.073714,1.014648,42.138205,7.641011,4.946610
min,0.000000e+00,59.647383,29.509349,0.000000,1.000000,1.000000,0.000000,1.400000,0.020000,1.000000
25%,4.400000e+06,59.863518,30.256876,1.000000,3.000000,9.000000,1.000000,37.600000,8.500000,1.000000
50%,5.950000e+06,59.939084,30.324338,2.000000,6.000000,13.000000,2.000000,51.000000,11.100000,1.000000
75%,8.800000e+06,59.999287,30.397938,3.000000,11.000000,20.000000,2.000000,69.000000,15.150000,11.000000
max,1.945383e+09,60.241984,30.711428,5.000000,35.000000,38.000000,9.000000,7856.000000,1272.000000,11.000000


In [92]:
MIN_AREA = 15  # Outlier range for floor area
MAX_AREA = 300
MIN_AREA = 10
MIN_KITCHEN = 3  # Outlier range for kitchen area
MAX_KITCHEN = 70

MIN_PRICE = 1_000_000  # Outlier range for price
MAX_PRICE = 100_000_000


def clean_data(df) -> pd.DataFrame:
    """Function removes excess columns and enforces
    correct data types.
    :param input_path: path to data
    :param output_path: path to save data
    :return: Updated DataFrame
    """
    
    df['date'] = pd.to_datetime(df['date'])
    # Column actually contains -1 and -2 values presumably for studio apartments.
    df['rooms'] = df['rooms'].apply(lambda x: 0 if x < 0 else x)
    df['price'] = df['price'].abs()  # Fix negative values
    # Drop price and area outliers.
    df = df[(df['area'] <= MAX_AREA) & (df['area'] >= MIN_AREA)]
    df = df[(df['price'] <= MAX_PRICE) & (df['price'] >= MIN_PRICE)]
    # Fix kitchen area outliers.
    # At first, replace all outliers with 0.
    df.loc[(df['kitchen_area'] >= MAX_KITCHEN) | (df['kitchen_area'] <= MIN_KITCHEN), 'kitchen_area'] = 0
    # Then calculate kitchen area based on the floor area, except for studios.
    area_mean, kitchen_mean = df[['area', 'kitchen_area']].quantile(0.5)
    kitchen_share = kitchen_mean / area_mean
    df.loc[(df['kitchen_area'] == 0) & (df['rooms'] != 0), 'kitchen_area'] = \
        df.loc[(df['kitchen_area'] == 0) & (df['rooms'] != 0), 'area'] * kitchen_share

    return df

In [93]:
df = clean_data(df)
df.shape

(460155, 11)

In [94]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    # Replace "date" with numeric features for year and month.
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df.drop('date', axis=1, inplace=True)
    # Apartment floor in relation to total number of floors.
    df['level_to_levels'] = df['level'] / df['levels']
    # Average size of room in the apartment.
    df['area_to_rooms'] = (df['area'] / df['rooms']).abs()
    # Fix division by zero.
    df.loc[df['area_to_rooms'] == np.inf, 'area_to_rooms'] = \
        df.loc[df['area_to_rooms'] == np.inf, 'area']
    return df

In [95]:
df = add_features(df)

In [96]:
df.shape

(460155, 14)